<a href="https://colab.research.google.com/github/daonly/2023Autumn/blob/main/%5B%EB%94%A5%EB%9F%AC%EB%8B%9D%5D%20%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8)_5%E1%84%8C%E1%85%A1%E1%86%BC_%E1%84%8B%E1%85%A9%E1%84%8E%E1%85%A1%E1%84%8B%E1%85%A7%E1%86%A8%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%91%E1%85%A1%E1%84%87%E1%85%A5%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 단순한 계층 구현하기

### 곱셈 계층

In [5]:
class MulLayer:
    def __init__(self):   # 일단 빈 걸로 정의해주기
        self.x = None
        self.y = None

    def forward(self, x, y):   # 이 class에 x와 y를 집어넣으면 곱을 리턴해줌
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):   # back propagation
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [7]:
#개념적으로 하면 뭐 이렇게 진행된다는 거지 실제로 이렇게 쓰이진 않음

apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple,apple_num)  #개수와 가격을 넣었고
price = mul_tax_layer.forward(apple_price,tax)   #세금을 덧붙이는 게지

print(price) # 220

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax) # 2.2 110 200

220.00000000000003
2.2 110.00000000000001 200


### 덧셈 계층

In [3]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [4]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 활성화 함수 계층 구현하기

### ReLU 계층

In [8]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)   # T / F로 변환
        out = x.copy()   # copy를 해둬야 x값이 계속 안 바뀐다아
        out[self.mask] = 0   # mask가 true면 0으로 바꿔줘라아ㅏ

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [9]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### Sigmoid 계층

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x) # 순전파의 출력을 out에 보관한 후 역전파 계산 때 그 값을 사용
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## Affine/Softmax 계층 구현하기

### Affine 계층

In [11]:
# 내적을 하는 과정이다~

X = np.random.rand(2) # 입력
W = np.random.rand(2,3) # 가중치
B = np.random.rand(3) # 편향

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


### 배치용 Affine 계층

In [12]:
# 순전파 때의 편향 덧셈
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)   #bias 크기를 일일이 맞춰줄 필요 없이, 한 단위를 쓰면 그 값을 더해줄 수 있...다...?

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [13]:
# 역전파 때의 편향 덧셈
dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [14]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

### Softmax-with-Loss 계층

In [15]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)  # sf를 거친 값과 target 간의 차이를 구하는 과정

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]  # 둘의 크기가 같다면
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()   # 둘의 크기가 다르다면
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

## 오차역전파법 구현하기

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# folder = '딥러닝의원리와응용'
folder = ''

### 오차역전파법을 적용한 신경망 구현하기

In [23]:
import sys, os
sys.path.append(f'/content/drive/MyDrive/{folder}')
# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화. i.e. 파라미터를 저장할 공간을 할당해주는 것.
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()  # 이 안에 forward 함수가 알아서 내장되어 있을 것.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()  # 음수 값들을 다 0으로 정리
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()   # 값들을 0-9로 매핑해두고, 그걸 정답이랑 비교.

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)  # Affine2까지 돌리고 나온 Y라는 결과에 sf를 적용한 후, 실제 정답과 cross-entropy를 하는 과정.

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)   # 어떤 값이 제일 큰지 보고, 그 값에만 1을 할당.
        if t.ndim != 1 : t = np.argmax(t, axis=1)   # 정답 레이블의 크기가 안 맞는 경우의 오류 방지

        accuracy = np.sum(y == t) / float(x.shape[0])  # batch size로 나눠달라
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):  # 실질적으로 grident 과정을 진행하는 단계
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward의 값을 1에서 업데이트하는 과정
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()   # 앞뒤를 반전시키고, 거기서 순차적으로 꺼내기
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 오차역전파법으로 구한 기울기 검증하기

In [24]:
import sys, os
sys.path.append(f'/content/drive/MyDrive/{folder}')
# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)   #ouput size는 실제 label 크기와 맞춰줘야 함.

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:5.447634715812627e-10
b1:3.2076802466778598e-09
W2:7.443079301985839e-09
b2:1.395841346521487e-07


### 오차역전파법을 사용한 학습 구현하기

In [25]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)   # 사용할 데이터를 랜덤하게 선택
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 갱신 (i.e. 업데이트)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.16988333333333333 0.1752
0.9014 0.9044
0.9213 0.9226
0.9338 0.9349
0.9441166666666667 0.9438
0.9498333333333333 0.9496
0.9549833333333333 0.9527
0.9604166666666667 0.9567
0.9630833333333333 0.959
0.9635333333333334 0.9603
0.9687 0.9634
0.97105 0.9646
0.9721666666666666 0.9647
0.9735833333333334 0.9664
0.9751833333333333 0.967
0.9755333333333334 0.9692
0.9781 0.9704
